# Pivot & Melt

## Kontingenční tabulka

Pivot table = kontingenční tabulka. 
Vytváření kontingenčních tabulek je proces ve kterém homogenní data agregujeme a jednotlivé "osy" agregace, 
resp. hodnoty v "osách" jsou určujícími pro řádky či sloupce.

Výchozí data rozdělujeme do podmnožin, velmi často disjunktních podmožin, nad kterými následně provádíme agragační operace. Mezi agregační operace patří `Count` `Sum` `Avg` `Max` `Min` apod. Grafické uspořádání je typické tím, že definiční filtry jsou v hlavičkách sloupců nebo hlavičkách řádků (první element ve sloupci či v řádku). Buňky jsou pak využity pro zobrazení výsledků agregace. Mnohdy je využíváno rozdělení do podmnožin podle více kritérií a vzniká tak "strom" podmínek rozdělující hlavní množinu do menších a menších podmonžin.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
sales = pd.read_csv(
    'sales_data.csv',
    parse_dates=['Date'])

In [5]:
sales.head()

Date  Day     Month  Year  Customer_Age       Age_Group  \
0 2013-11-26   26  November  2013            19     Youth (<25)   
1 2015-11-26   26  November  2015            19     Youth (<25)   
2 2014-03-23   23     March  2014            49  Adults (35-64)   
3 2016-03-23   23     March  2016            49  Adults (35-64)   
4 2014-05-15   15       May  2014            47  Adults (35-64)   

  Customer_Gender    Country             State Product_Category Sub_Category  \
0               M     Canada  British Columbia      Accessories   Bike Racks   
1               M     Canada  British Columbia      Accessories   Bike Racks   
2               M  Australia   New South Wales      Accessories   Bike Racks   
3               M  Australia   New South Wales      Accessories   Bike Racks   
4               F  Australia   New South Wales      Accessories   Bike Racks   

               Product  Order_Quantity  Unit_Cost  Unit_Price  Profit  Cost  \
0  Hitch Rack - 4-Bike               8         45         120     590   360   
1  Hitch Rack - 4-Bike               8         45         120     590   360   
2  Hitch Rack - 4-Bike              23         45         120    1366  1035   
3  Hitch Rack - 4-Bike              20         45         120    1188   900   
4  Hitch Rack - 4-Bike               4         45         120     238   180   

   Revenue  
0      950  
1      950  
2     2401  
3     2088  
4      418

In [6]:
import json
data = json.loads(sales.to_json(orient="table"))
print(data.keys())
data = data["data"]
print(len(data))
print(data[0])

dict_keys(['schema', 'data'])
113036
{'index': 0, 'Date': '2013-11-26T00:00:00.000', 'Day': 26, 'Month': 'November', 'Year': 2013, 'Customer_Age': 19, 'Age_Group': 'Youth (<25)', 'Customer_Gender': 'M', 'Country': 'Canada', 'State': 'British Columbia', 'Product_Category': 'Accessories', 'Sub_Category': 'Bike Racks', 'Product': 'Hitch Rack - 4-Bike', 'Order_Quantity': 8, 'Unit_Cost': 45, 'Unit_Price': 120, 'Profit': 590, 'Cost': 360, 'Revenue': 950}


Výběr hodnot v ose

|           | podmínka A | podmínka B | podmínka C |
|-----------|:----------:|:----------:|:----------:|
| podmínka 1|  P1 & PA   | P1 & PB    |  P1 & PC   |
| podmínka 2|  P2 & PA   | P2 & PB    |  P2 & PC   |


Obvykle u kontingenční tabulky platí, že pro položku X je pravdivý pouze jeden z výroků 
$$PA(X) \wedge PB(X) \wedge PC(X) = False$$

obdobně

$$P1(X) \wedge P2(X) = False$$

Tabulka může mít i jinou podobu

|            |           |            |
|------------|-----------|:----------:|
| podmínka A | podmínka 1|  P1 & PA   |
| podmínka A | podmínka 2|  P2 & PA   |
| podmínka B | podmínka 1|  P1 & PB   |
| podmínka B | podmínka 2|  P2 & PB   |
| podmínka C | podmínka 1|  P1 & PC   |
| podmínka C | podmínka 2|  P2 & PC   |

případně

|            |           |            |
|------------|-----------|:----------:|
| podmínka 1 | podmínka A|  P1 & PA   |
| podmínka 1 | podmínka B|  P1 & PB   |
| podmínka 1 | podmínka C|  P1 & PC   |
| podmínka 2 | podmínka A|  P2 & PA   |
| podmínka 2 | podmínka B|  P2 & PB   |
| podmínka 2 | podmínka C|  P2 & PC   |

V případě většího počtu os (disjuktních sad podmínek) dochází k vnoření podmínek

|           |           | podmínka A | podmínka B | podmínka C |
|-----------|-----------|:----------:|:----------:|:----------:|
| podmínka α| podmínka 1|  Pα & P1 & PA   | Pα & P1 & PB |  Pα & P1 & PC |
| podmínka α| podmínka 2|  Pα & P2 & PA   | Pα & P2 & PB |  Pα & P2 & PC |
| podmínka β| podmínka 1|  Pβ & P1 & PA   | Pβ & P1 & PB |  Pβ & P1 & PC |
| podmínka β| podmínka 2|  Pβ & P2 & PA   | Pβ & P2 & PB |  Pβ & P2 & PC |

Vždy se očekává, že ve vnitřních buňkách je aplikován agregační operátor na podmnožinu splňující současně všechny podmínky identifikované polohou buňky.

Mezi tyto agregační operátory patří sum, count, avg apod. V některých specifických případech lze do buňky uvést záznamy splňující podmínky, ale toto je možné jen pro malé datové sady.

### Příklad 1

**Příklad**

Znáte strukturu jednoho záznamu. Vytvořte kód, který zjistí všechny možné hodnoty v ose "Product_Category" (jaké hodnoty tento atribut může nabývat).

In [132]:
axis = "Product_Category"
product_category_values = [] # upravte s cílem splnit zadání
print(product_category_values)

[]


**Řešení**

In [133]:
axis = "Product_Category"
product_category_values = set(map(lambda item: item[axis], data))
print(product_category_values)

{'Accessories', 'Clothing', 'Bikes'}


### Příklad 2A

Vytvořte funkci, která definuje filtr pro danou hodnotu attributu. Jestliže atribut řádku nabývá dané hodnoty, je vácena hodnota True, v opačném případě False.

In [ ]:
def createFilter(value):
    def result(row):
        #
        #
        #
    return result

row = data[0]
f = createFilter('Accessories')
print(f(row))

**Řešení**

In [138]:
def createFilter(value):
    def result(row):
        return row["Product_Category"] == value
    return result

row = data[0]
f = createFilter('Accessories')
print(f(row))

True


### Příklad 2

**Příklad**

Vytvořte funkce vracející "True" / "False" podle příslušnosti do skupiny v ose "Product_Category".

Ověřte si správné fungování na prvním záznamu. Hodnotu `True` mustí vrátit jediná funkce, zbývající musí vrátit hodnotu `False`. Pomocí funkcí rozdělujeme množinu všech dat do disjunktních podmnožin.

Funkce, které vytváříte jsou tzv. Closures. Za určitých okolností může vzniknout sada funkcí, které vrací stejné výsledky (všechny funkce vrací stejný výsledek pro všechny řádky v tabulce). Takové implementaci je nutné se vyhnout.

20 min.

In [134]:
product_category_selectors = dict()#zmente / naplnte, pouzijte vice radku

print(product_category_selectors)

row = data[0]
for key, value in product_category_selectors.items():
    print(key, value(row))

{}


**Řešení A**

In [136]:
def createFilter(value):
    def resultFilter(item):
        return item['Product_Category'] == value
    return resultFilter

product_category_selectors = {}
for row in data:
    attValue = row['Product_Category']
    if product_category_selectors.get(attValue, None) is None:
        currentFilter = createFilter(attValue)
        product_category_selectors[attValue] = currentFilter
    
print(product_category_selectors)

row = data[0]
for key, value in product_category_selectors.items():
    print(key, value(row))

{'Accessories': <function createFilter.<locals>.resultFilter at 0x7f6b8e145ab0>, 'Clothing': <function createFilter.<locals>.resultFilter at 0x7f6b8e1460e0>, 'Bikes': <function createFilter.<locals>.resultFilter at 0x7f6b8e145900>}
Accessories True
Clothing False
Bikes False


**Řešení B**

In [137]:
def createLambda(value):
    return lambda item: item["Product_Category"] == value

product_category_selectors = dict((value, createLambda(value)) for value in product_category_values)
print(product_category_selectors)

row = data[0]
for key, value in product_category_selectors.items():
    print(key, value(row))

{'Accessories': <function createLambda.<locals>.<lambda> at 0x7f6b8e144310>, 'Clothing': <function createLambda.<locals>.<lambda> at 0x7f6b9a619990>, 'Bikes': <function createLambda.<locals>.<lambda> at 0x7f6b9a618ee0>}
Accessories True
Clothing False
Bikes False


### Příklad 3

**Příklad**

Vytvořte funkci, jejímž výstupem je sada filtrů podle diskrétních hodnot u daného atributu (sloupce tabulky).
Jedná se o zobecnění předchozích úloh.

In [104]:
def createFilters(attributeName, table):
    def createLambda(value):
        return lambda item: item[attributeName] == value

    result = {}
    for row in table:
        #
        #
        #
        #
        pass
    return result

ageFilters = createFilters("Age_Group", data)
categoryFilters = createFilters("Product_Category", data)
print(ageFilters)
print(categoryFilters)

{}
{}


**Řešení**

In [105]:
def createFilters(attributeName, table):
    def createLambda(value):
        return lambda item: item[attributeName] == value

    result = {}
    for row in table:
        value = row[attributeName]
        if result.get(value, None) is None:
            result[value] = createLambda(value)
    return result

ageFilters = createFilters("Age_Group", data)
categoryFilters = createFilters("Product_Category", data)
print(ageFilters)
print(categoryFilters)

{'Youth (<25)': <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61a170>, 'Adults (35-64)': <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61a200>, 'Young Adults (25-34)': <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61a290>, 'Seniors (64+)': <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61a320>}
{'Accessories': <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61a440>, 'Clothing': <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61a4d0>, 'Bikes': <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61a560>}


### Příklad 4

**Příklad**

Kolik řádků v datech splňuje podmínku definovanou funkcí `product_category_selector`.

In [139]:
[(product_category_name, product_category_selector), *_] = product_category_selectors.items()
print(product_category_name, product_category_selector)

filteredData = [] # Doplnte
print(len(filteredData))

Accessories <function createLambda.<locals>.<lambda> at 0x7f6b8e144310>
70120


**Řešení A**

In [107]:
[(product_category_name, product_category_selector), *_] = product_category_selectors.items()
print(product_category_name, product_category_selector)

filteredData = []
for row in date:
    if product_category_selector(row):
        filteredData.append(row)

print(len(filteredData))

Accessories <function createLambda.<locals>.<lambda> at 0x7f6b9abd3250>
70120


**Řešení B**

In [142]:
[(product_category_name, product_category_selector), *_] = product_category_selectors.items()
print(product_category_name, product_category_selector)

filteredData = list(filter(product_category_selector, data))
print(len(filteredData))

Accessories <function createLambda.<locals>.<lambda> at 0x7f6b8e144310>
70120


**Řešení C**

In [139]:
[(product_category_name, product_category_selector), *_] = product_category_selectors.items()
print(product_category_name, product_category_selector)

filteredData = [row for row in data if product_category_selector(row)] # Doplnte
print(len(filteredData))

Accessories <function createLambda.<locals>.<lambda> at 0x7f6b8e144310>
70120


### Příklad 5

**Příklad**

Zjistěte součet attributu "Profit" pro řádky splňující podmínku definovanou funkcí.

In [108]:
[(product_category_name, product_category_selector), *_] = product_category_selectors.items()
print(product_category_name, product_category_selector)

filteredData = list(filter(product_category_selector, data))
total = 0 # Doplnte, pripadne navrhnete jine reseni
print(total)

Accessories <function createLambda.<locals>.<lambda> at 0x7f6b9abd3250>
0


**Řešení A**

In [144]:
[(product_category_name, product_category_selector), *_] = product_category_selectors.items()
print(product_category_name, product_category_selector)

total = 0
for row in data:
    if product_category_selector(row):
        value = row["Profit"]
        total = total + value

print(total)

Accessories <function createLambda.<locals>.<lambda> at 0x7f6b8e144310>
8862377


**Řešení B**

In [143]:
[(product_category_name, product_category_selector), *_] = product_category_selectors.items()
print(product_category_name, product_category_selector)

filteredData = list(filter(product_category_selector, data))
mapped = map(lambda item: item["Profit"], filteredData)
total = sum(mapped)

print(total)

Accessories <function createLambda.<locals>.<lambda> at 0x7f6b8e144310>
8862377


**Řešení C**

In [143]:
[(product_category_name, product_category_selector), *_] = product_category_selectors.items()
print(product_category_name, product_category_selector)

filteredData = (filter(product_category_selector, data))
mapped = map(lambda item: item["Profit"], filteredData)
total = sum(mapped)

print(total)

Accessories <function createLambda.<locals>.<lambda> at 0x7f6b8e144310>
8862377


### Příklad 6

**Příklad**

Vytvořte funkci, která vytvoří funkci počítající sumu nad attributem pro řádky splňující podmínku.

In [110]:
def createSumarizer(attributeName, filterFunc):
    def result(data):
        filtered = [] #zmente
        mapped = [] #zmente
        total = sum(mapped)
        return total
    return result

[(product_category_name, product_category_selector), *_] = product_category_selectors.items()
print(product_category_name, product_category_selector)
sumarizer = createSumarizer("Profit", product_category_selector)
total = sumarizer(data)
print(total)

Accessories <function createLambda.<locals>.<lambda> at 0x7f6b9abd3250>
0


**Řešení**

In [111]:
def createSumarizer(attributeName, filterFunc):
    def result(data):
        filtered = filter(filterFunc, data)
        mapped = map(lambda item: item[attributeName], filtered)
        total = sum(mapped)
        return total
    return result

[(product_category_name, product_category_selector), *_] = product_category_selectors.items()
print(product_category_name, product_category_selector)
sumarizer = createSumarizer("Profit", product_category_selector)
total = sumarizer(data)
print(total)

Accessories <function createLambda.<locals>.<lambda> at 0x7f6b9abd3250>
8862377


### Příklad 7

**Příklad**

Ze seznamu filtrů vytvořte sumarizery a použijte výsledek na data

In [112]:
sumarizers = dict()#upravte

print(sumarizers)

print('-'*30)
for key, value in sumarizers.items():
    print(f"{key:10}\t{value(data)}")


{}
------------------------------


**Řešení**

In [146]:
sumarizers = dict((key, createSumarizer("Profit", filterfunc)) for key, filterfunc in product_category_selectors.items())
print(sumarizers)

print('-'*30)
for key, value in sumarizers.items():
    print(f"{key:10}\t{value(data)}")


{'Accessories': <function createSumarizer.<locals>.result at 0x7f6b9a61a8c0>, 'Clothing': <function createSumarizer.<locals>.result at 0x7f6b9a619ab0>, 'Bikes': <function createSumarizer.<locals>.result at 0x7f6b9a61add0>}
------------------------------
Accessories	8862377
Clothing  	2839447
Bikes     	20519276


Výše uvedený výsledek je možné považovat za nejjednodušší kontingenční tabulku

### Příklad 8

**Příklad**

Seznamte se s funkcí `product` z knihovny `itertools`

In [114]:
import itertools

prod = list(itertools.product(['A', 'B'], ['1', '2']))
print(prod)

[('A', '1'), ('A', '2'), ('B', '1'), ('B', '2')]


### Příklad 9

**Příklad**

Použijte `product` na filtrech.

In [115]:
import itertools

ageFilters = createFilters("Age_Group", data)
categoryFilters = createFilters("Product_Category", data)

nestedNames = list(itertools.product(ageFilters.keys(), categoryFilters.keys()))
print(nestedNames)

[('Youth (<25)', 'Accessories'), ('Youth (<25)', 'Clothing'), ('Youth (<25)', 'Bikes'), ('Adults (35-64)', 'Accessories'), ('Adults (35-64)', 'Clothing'), ('Adults (35-64)', 'Bikes'), ('Young Adults (25-34)', 'Accessories'), ('Young Adults (25-34)', 'Clothing'), ('Young Adults (25-34)', 'Bikes'), ('Seniors (64+)', 'Accessories'), ('Seniors (64+)', 'Clothing'), ('Seniors (64+)', 'Bikes')]


In [116]:
import itertools

ageFilters = createFilters("Age_Group", data)
categoryFilters = createFilters("Product_Category", data)

nestedNames = list(itertools.product(ageFilters.items(), categoryFilters.items()))
print(nestedNames)

[(('Youth (<25)', <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61b010>), ('Accessories', <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b8e145ab0>)), (('Youth (<25)', <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61b010>), ('Clothing', <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b8e145990>)), (('Youth (<25)', <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61b010>), ('Bikes', <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61b760>)), (('Adults (35-64)', <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61a560>), ('Accessories', <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b8e145ab0>)), (('Adults (35-64)', <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f6b9a61a560>), ('Clothing', <function createFilters.<locals>.createLambda.<locals>.<lambda> at 0x7f

### Příklad 10

**Příklad**

Vytvořte vnořené filtry. Vnořený filtr vzniká jako `And` nad oběma filtry.

In [117]:
import itertools

def A(filterA, filterB):
    return lambda item: filterA(item) and filterB(item)

def A(*filters):
    def result_func(item):
        result = True
        for f in filters:
            if not (f(item)):
                result = False
                break
        return result
    return result_func

def createNestedFilters(data, attributeNameA, attributeNameB):
    filtersA = createFilters(attributeNameA, data)
    categoryFilters = createFilters(attributeNameB, data)
    nestedNames = list(itertools.product(filtersA.items(), categoryFilters.items()))
    
    result = {}
    for nestedName in nestedNames:
        filters = []
        names = []
        for name, function in nestedName:
            names.append(name)
            filters.append(function)
        level = result
        for name in names[:-1]:
            sublevel = level.get(name, None)
            if sublevel is None:
                sublevel = {}
                level[name] = sublevel
            level = sublevel
                
        masterfilter = A(*filters)
        name = names[-1]
        level[name] = masterfilter
    return result

nestedFilters = createNestedFilters(data, "Age_Group", "Product_Category")
print(nestedFilters)

{'Youth (<25)': {'Accessories': <function A.<locals>.result_func at 0x7f6b8e1cde10>, 'Clothing': <function A.<locals>.result_func at 0x7f6b9a61ba30>, 'Bikes': <function A.<locals>.result_func at 0x7f6b9a61bac0>}, 'Adults (35-64)': {'Accessories': <function A.<locals>.result_func at 0x7f6b9a61bb50>, 'Clothing': <function A.<locals>.result_func at 0x7f6b9a61bbe0>, 'Bikes': <function A.<locals>.result_func at 0x7f6b9a61bc70>}, 'Young Adults (25-34)': {'Accessories': <function A.<locals>.result_func at 0x7f6b9a61bd00>, 'Clothing': <function A.<locals>.result_func at 0x7f6b9a61bd90>, 'Bikes': <function A.<locals>.result_func at 0x7f6b9a61be20>}, 'Seniors (64+)': {'Accessories': <function A.<locals>.result_func at 0x7f6b9a61beb0>, 'Clothing': <function A.<locals>.result_func at 0x7f6b9a61bf40>, 'Bikes': <function A.<locals>.result_func at 0x7f6b841a0040>}}


In [118]:
def applyNestedFilters(filters, data):
    def inner(prefixes, filters, data):
        for key, f in filters.items():
            if isinstance(f, dict):
                for item in inner([*prefixes, key], f, data):
                    yield item
            else:
                yield [*prefixes, key], filter(f, data)
    return inner([], filters, data)
                
nestedFilters = createNestedFilters(data, "Age_Group", "Product_Category")
groupedData = list(applyNestedFilters(nestedFilters, data))
for prefixes, dataset in groupedData :
    #print(''.join(prefixes), sum(dataset))
    #print(prefixes, dataset)
    total = sum(map(lambda item: item["Profit"], dataset))
    print('\t'.join(prefixes), '\t', total)


Youth (<25)	Accessories 	 1255267
Youth (<25)	Clothing 	 410079
Youth (<25)	Bikes 	 2709246
Adults (35-64)	Accessories 	 4611837
Adults (35-64)	Clothing 	 1527164
Adults (35-64)	Bikes 	 10182581
Young Adults (25-34)	Accessories 	 2921240
Young Adults (25-34)	Clothing 	 884614
Young Adults (25-34)	Bikes 	 7580907
Seniors (64+)	Accessories 	 74033
Seniors (64+)	Clothing 	 17590
Seniors (64+)	Bikes 	 46542


Předchozí výstup je kontingenční tabulkou. Kontingenční tabulky je možno uspořádávat různě a také je možné mít v kontingenční tabulce více atributů (zde jsou dva).

In [119]:
import itertools

ageFilters = createFilters("Age_Group", data)
categoryFilters = createFilters("Product_Category", data)

nestedNames = list(itertools.product(ageFilters.keys(), categoryFilters.keys()))
print(nestedNames)

[('Youth (<25)', 'Accessories'), ('Youth (<25)', 'Clothing'), ('Youth (<25)', 'Bikes'), ('Adults (35-64)', 'Accessories'), ('Adults (35-64)', 'Clothing'), ('Adults (35-64)', 'Bikes'), ('Young Adults (25-34)', 'Accessories'), ('Young Adults (25-34)', 'Clothing'), ('Young Adults (25-34)', 'Bikes'), ('Seniors (64+)', 'Accessories'), ('Seniors (64+)', 'Clothing'), ('Seniors (64+)', 'Bikes')]


### Pandas

In [7]:
import pandas as pd

dfData = pd.DataFrame(data)
dfData

index                     Date  Day     Month  Year  Customer_Age  \
0            0  2013-11-26T00:00:00.000   26  November  2013            19   
1            1  2015-11-26T00:00:00.000   26  November  2015            19   
2            2  2014-03-23T00:00:00.000   23     March  2014            49   
3            3  2016-03-23T00:00:00.000   23     March  2016            49   
4            4  2014-05-15T00:00:00.000   15       May  2014            47   
...        ...                      ...  ...       ...   ...           ...   
113031  113031  2016-04-12T00:00:00.000   12     April  2016            41   
113032  113032  2014-04-02T00:00:00.000    2     April  2014            18   
113033  113033  2016-04-02T00:00:00.000    2     April  2016            18   
113034  113034  2014-03-04T00:00:00.000    4     March  2014            37   
113035  113035  2016-03-04T00:00:00.000    4     March  2016            37   

             Age_Group Customer_Gender         Country             State  \
0          Youth (<25)               M          Canada  British Columbia   
1          Youth (<25)               M          Canada  British Columbia   
2       Adults (35-64)               M       Australia   New South Wales   
3       Adults (35-64)               M       Australia   New South Wales   
4       Adults (35-64)               F       Australia   New South Wales   
...                ...             ...             ...               ...   
113031  Adults (35-64)               M  United Kingdom           England   
113032     Youth (<25)               M       Australia        Queensland   
113033     Youth (<25)               M       Australia        Queensland   
113034  Adults (35-64)               F          France     Seine (Paris)   
113035  Adults (35-64)               F          France     Seine (Paris)   

       Product_Category Sub_Category              Product  Order_Quantity  \
0           Accessories   Bike Racks  Hitch Rack - 4-Bike               8   
1           Accessories   Bike Racks  Hitch Rack - 4-Bike               8   
2           Accessories   Bike Racks  Hitch Rack - 4-Bike              23   
3           Accessories   Bike Racks  Hitch Rack - 4-Bike              20   
4           Accessories   Bike Racks  Hitch Rack - 4-Bike               4   
...                 ...          ...                  ...             ...   
113031         Clothing        Vests      Classic Vest, S               3   
113032         Clothing        Vests      Classic Vest, M              22   
113033         Clothing        Vests      Classic Vest, M              22   
113034         Clothing        Vests      Classic Vest, L              24   
113035         Clothing        Vests      Classic Vest, L              23   

        Unit_Cost  Unit_Price  Profit  Cost  Revenue  
0              45         120     590   360      950  
1              45         120     590   360      950  
2              45         120    1366  1035     2401  
3              45         120    1188   900     2088  
4              45         120     238   180      418  
...           ...         ...     ...   ...      ...  
113031         24          64     112    72      184  
113032         24          64     655   528     1183  
113033         24          64     655   528     1183  
113034         24          64     684   576     1260  
113035         24          64     655   552     1207  

[113036 rows x 19 columns]

In [15]:
pd.pivot_table(dfData, index="Age_Group", columns=["Product_Category"], values="Profit", aggfunc=sum) #1255267

Product_Category      Accessories     Bikes  Clothing
Age_Group                                            
Adults (35-64)            4611837  10182581   1527164
Seniors (64+)               74033     46542     17590
Young Adults (25-34)      2921240   7580907    884614
Youth (<25)               1255267   2709246    410079

In [16]:
pd.pivot_table(dfData, columns=["Age_Group", "Product_Category"], values="Profit", aggfunc=sum)

Age_Group        Adults (35-64)                    Seniors (64+)         \
Product_Category    Accessories     Bikes Clothing   Accessories  Bikes   
Profit                  4611837  10182581  1527164         74033  46542   

Age_Group                 Young Adults (25-34)                   Youth (<25)  \
Product_Category Clothing          Accessories    Bikes Clothing Accessories   
Profit              17590              2921240  7580907   884614     1255267   

Age_Group                           
Product_Category    Bikes Clothing  
Profit            2709246   410079

## "Dlouhá data"

https://towardsdatascience.com/reshape-pandas-dataframe-with-melt-in-python-tutorial-and-visualization-29ec1450bb02

In [120]:
data = [
    {'name': 'Anna', 'subject': 'Informatics', 'semester1': 'A', 'semester2': 'B', 'semester3': 'A'},
    {'name': 'Betty', 'subject': 'Informatics', 'semester1': 'C', 'semester2': 'A', 'semester3': 'A'},
    {'name': 'Cindy', 'subject': 'Mathematics', 'semester1': 'C', 'semester2': 'D', 'semester3': 'D'},
    {'name': 'Dita', 'subject': 'Biology', 'semester1': 'A', 'semester2': 'A', 'semester3': 'C'},
]

In [121]:
keys = {}
for row in data:
    for name in row.keys():
        keys[name] = True
        
for key in keys:
    print(f"{key:15}", end="")
print()
for row in data:
    for key in keys:
        value = row.get(key, None)
        print(f"{value:15}", end="")
    print()

name           subject        semester1      semester2      semester3      
Anna           Informatics    A              B              A              
Betty          Informatics    C              A              A              
Cindy          Mathematics    C              D              D              
Dita           Biology        A              A              C              


**Příklad**

Předpokládejte, že tabulka výše je kontingenční tabulkou. Vytvořte zdrojová data.

In [122]:
def ToLongForm(data, indexes=[], varName="variable", valueName="value"):
    for item in data:
        for key in item.keys():
            row = {}
            ################
            ################
            ################
            ################
            ################
            ################
            ################
            ################
            yield row
        
        
longData = list(ToLongForm(data, indexes=['name', 'subject'], varName='semester', valueName='grade'))
print(longData)

[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]


**Řešení**

In [123]:
def ToLongForm(data, indexes=[], varName="variable", valueName="value"):
    for item in data:
        for key in item.keys():
            row = {}
            for i in indexes:
                row[i] = item.get(i, None)
                
            if key in indexes:
                continue
            row[varName] = key
            row[valueName] = item[key]
            yield row
        
longData = list(ToLongForm(data, indexes=['name', 'subject'], varName='semester', valueName='grade'))
print(longData)

[{'name': 'Anna', 'subject': 'Informatics', 'semester': 'semester1', 'grade': 'A'}, {'name': 'Anna', 'subject': 'Informatics', 'semester': 'semester2', 'grade': 'B'}, {'name': 'Anna', 'subject': 'Informatics', 'semester': 'semester3', 'grade': 'A'}, {'name': 'Betty', 'subject': 'Informatics', 'semester': 'semester1', 'grade': 'C'}, {'name': 'Betty', 'subject': 'Informatics', 'semester': 'semester2', 'grade': 'A'}, {'name': 'Betty', 'subject': 'Informatics', 'semester': 'semester3', 'grade': 'A'}, {'name': 'Cindy', 'subject': 'Mathematics', 'semester': 'semester1', 'grade': 'C'}, {'name': 'Cindy', 'subject': 'Mathematics', 'semester': 'semester2', 'grade': 'D'}, {'name': 'Cindy', 'subject': 'Mathematics', 'semester': 'semester3', 'grade': 'D'}, {'name': 'Dita', 'subject': 'Biology', 'semester': 'semester1', 'grade': 'A'}, {'name': 'Dita', 'subject': 'Biology', 'semester': 'semester2', 'grade': 'A'}, {'name': 'Dita', 'subject': 'Biology', 'semester': 'semester3', 'grade': 'C'}]
